In [7]:
#import tensorflow as tf

In [1]:
import h5py as h5

In [9]:
#tf.reset_default_graph()


# modelDir='knee_trained_MoDL'

# loadChkPoint=tf.train.latest_checkpoint(modelDir)

In [9]:
with h5.File('/home/hpc/iwbi/iwbi009h/CIP_-MoDL_vs_VarNet-/test/file_brain_AXT1_201_6002695.h5','r') as f:
    print(f.keys())
    print(f['reconstruction_rss'])
    print(f['kspace'])
    with h5.File('test.h5', 'w') as target:
        target.create_dataset('atb', data = f['reconstruction_rss'])
        target.create_dataset('org', data = f['kspace'])
        print(target.keys())

<KeysViewHDF5 ['ismrmrd_header', 'kspace', 'reconstruction_rss']>
<HDF5 dataset "reconstruction_rss": shape (12, 320, 320), type "<f4">
<HDF5 dataset "kspace": shape (12, 20, 640, 320), type "<c8">
<KeysViewHDF5 ['atb', 'org']>


In [5]:
with h5.File('knee_demo_data.h5','r') as f:
    print(f.keys())
    print(f['atb'])
    print(f['csm'])

<KeysViewHDF5 ['atb', 'csm', 'mask', 'org']>
<HDF5 dataset "atb": shape (1, 640, 368), type "<c8">
<HDF5 dataset "csm": shape (1, 15, 640, 368), type "<c8">


In [ ]:
with h5.File('demoImage.hdf5','r') as f:
    print(f.keys())
    print(f['tstMask'])